<a href="https://colab.research.google.com/github/sj0054pe/MBTI_Estim/blob/master/src/2_Twitter_%E5%90%84%E3%83%A6%E3%83%BC%E3%82%B6%E3%83%BC%E3%81%AETweet%E3%82%92%E9%9B%86%E3%82%81%E3%82%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#<center>Twitter 各ユーザーのTweetを集める</center>

#0. 作業環境の準備

##0.1. Google Driveをマウント

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

##0.2. カレントディレクトリへ移動

In [ ]:
%cd /content/drive/MyDrive/研究室/研究_M2/src

/content/drive/MyDrive/研究室/研究_M2/src


##0.3. 現在日時の取得

In [ ]:
import datetime
from pytz import timezone

def Pull_Datetime():
    dates=datetime.datetime.now(timezone('Asia/Tokyo'))
    exept_microsec=dates.strftime("%Y-%m-%d-%H-%M")
    print('[Today : %s]' % str(exept_microsec))

    DATETIME=exept_microsec
    return DATETIME

DATETIME=Pull_Datetime()

[Today : 2021-08-24-23-57]


##0.4. 作成途中のデータがあればこちらを動かす。

In [ ]:
#DATETIME = '2021-08-24-23-57'

# OUTPUT_PATH =  './Assets/Assets_Output/'
# OUTPUT_DIR=OUTPUT_PATH+ "PersonalEstim_%s/" % DATETIME
# OUTPUT_DIR_RESULTS=OUTPUT_PATH+ "PersonalEstim_%s/results/" % DATETIME

# KEYWORDS_TypeMBTI = '2017-2018'
# KEYWORDS_FileName='2017'

##0.5. 作業ディレクトリの作成

In [ ]:
import os
OUTPUT_PATH =  './Assets/Assets_Output/'
OUTPUT_DIR=OUTPUT_PATH+ "PersonalEstim_%s/" % DATETIME
OUTPUT_DIR_RESULTS=OUTPUT_PATH+ "PersonalEstim_%s/results/" % DATETIME

KEYWORDS_TypeMBTI = '2017-2018'

try:
  os.mkdir(OUTPUT_DIR)
except:
  print(OUTPUT_DIR, '    already exists')
  print()
try:
  os.mkdir(OUTPUT_DIR_RESULTS)
except:
  print(OUTPUT_DIR_RESULTS, '    the directory already exists')
  print()

##0.6. ファイル名に含む単語 (任意) を設定する。

ファイルの中身を明示する内容がオススメ。

In [ ]:
 KEYWORDS_FileName='2017'

#2. データの準備

##2.1. 性格診断結果を投稿したデータの読み込み

個人の特定に繫がるtweet_idは伏せてます。

In [3]:
import pandas as pd

df_2017=pd.read_csv('/content/drive/MyDrive/研究室/研究_M2/src/Assets/Assets_Output/PersonalEstim_2021-08-24-00-36/results/UserListFinal_2017.csv', engine='python')
df_2017=df_2017.dropna(how='any')
df_2017=df_2017.sort_values('tweet_time')
list_AuthorID=df_2017['author_id'].values.tolist()
#list_AuthorID=list(map(int, list_AuthorID))
print("ユーザー数 : ", len(list_AuthorID))

df_2017[['text', 'tweet_time', 'lang']].head()

36244


,text,tweet_time,lang
36265,私の性格タイプは “冒険家” です。あなたのタイプは何ですか？ https://t.co/o...,2017-01-01 04:44:24+00:00,ja
36264,私の性格タイプは “擁護者” です。あなたのタイプは何ですか？ https://t.co/S...,2017-01-01 07:15:52+00:00,ja
36263,私の性格タイプは “エンターテイナー” です。あなたのタイプは何ですか？ https://t...,2017-01-01 08:08:09+00:00,ja
36262,私の性格タイプは “建築家” です。あなたのタイプは何ですか？ https://t.co/O...,2017-01-01 11:42:36+00:00,ja
36261,私の性格タイプは “論理学者” です。あなたのタイプは何ですか？ https://t.co/...,2017-01-01 12:15:14+00:00,ja


##2.2. 重複するユーザーの統一

In [4]:
#print(list_AuthorID)
set_AuthorID=set(list_AuthorID)
#print(type(set_AuthorID))
#print(len(set_AuthorID))

list_AuthorID = [x for x in set_AuthorID]
#print(type(list_AuthorID))

list_AuthorID=list(map(int, list_AuthorID))
#print(list_AuthorID)
print("統一後 : ", len(list_AuthorID))

統一後 :  33955


#3. Twitter APIを利用して、特定のユーザーが過去に発信した投稿を取得する。

##3.1. 「Twiiter APIのエンドポイントを作成」する関数

今回のTwitter API のURLは「特定のユーザーが過去に発信した投稿を取得」するエンドポイントである。

In [ ]:
import yaml
import os # For saving access tokens and for file management when creating and adding to the dataset
import requests # For sending GET requests from the API
import Twitter_API_Key

def Create_url_UserTimeline(list_AuthorID, start_date, end_date, max_results = 10):
    #Change to the endpoint you want to collect data from
    search_url = "https://api.twitter.com/2/users/%s/tweets" % list_AuthorID
    #change params based on the endpoint you are using
    query_params = {'max_results': max_results,
                    'expansions': 'author_id,in_reply_to_user_id,geo.place_id',
                    'tweet.fields': 'id,text,author_id,in_reply_to_user_id,geo,conversation_id,created_at,lang,public_metrics,referenced_tweets,reply_settings,source',
                    'user.fields': 'id,name,username,created_at,description,public_metrics,verified',
                    'place.fields': 'full_name,id,country,country_code,geo,name,place_type',
                    'next_token': {},
                    'start_time': start_date,
                    'end_time': end_date
                    }
                    
    return (search_url, query_params)

##3.2. 「エンドポイントを利用してTwitter APIへアクセス」する関数

In [ ]:
def Connect_to_endpoint(url, params, next_token = None):
    #prepare params
    params['next_token'] = next_token   #params object received from create_url function
    #pull bearer token
    bearer_token = Twitter_API_Key.Auth_FullAchieve()
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    #request Twtitter
    response = requests.request("GET", url, headers = headers, params = params)
    print("Endpoint Response Code: " + str(response.status_code))
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
        
    return response.json()

##3.3. 「特定のユーザーが過去に発信した投稿を取得」する関数

In [1]:
import dateutil.parser
import pandas as pd
import tqdm

def Pull_tweets_from_specific_users(list_AuthorID, start_time, end_time, max_results, next_token, OUTPUT_DIR_RESULTS, KEYWORDS_TypeMBTI):
  url = Create_url_UserTimeline(list_AuthorID, start_time, end_time, max_results)
  print("url : ", url)
  json_UserTimeline= Connect_to_endpoint(url[0], url[1], next_token)

  json_UserTimeline
  print(json_UserTimeline)
  list_dict_keys=list(json_UserTimeline.keys())
  if list_dict_keys[0]=='errors':
    return 0
  elif json_UserTimeline['meta']['result_count']==0:
    return 0
  else: 
    list_EachData=json_UserTimeline['data']

  list_csv=[]
  list_csv.clear()
  list_header=['author_id', 'text',  'tweet_id','tweet_time', 'conversation_id','lang']
  for element in list_EachData:
    #print(element['text'])
    tweet_AuthorID=element['author_id']
    tweet_Texts=element['text']
    tweet_id = element['id']
    tweet_time = dateutil.parser.parse(element['created_at'])
    convarsation_id = element['conversation_id']
    tweet_lang = element['lang']

    list_sub_row=[tweet_AuthorID, tweet_Texts, tweet_id, tweet_time, convarsation_id, tweet_lang]
    list_csv.append(list_sub_row)

  df_UserTimeline=pd.DataFrame(list_csv, columns=list_header)
  df_UserTimeline.to_csv('%sTweetsByOneUser_%s_%s.csv' % (OUTPUT_DIR_RESULTS, KEYWORDS_TypeMBTI, df_UserTimeline['author_id'][0]))

  return df_UserTimeline

##3.5 特定のユーザーが過去に発信した投稿を取得する。

これまで作成した関数を動かす。

In [ ]:
next_token=None
max_results_author=100
start_time = "2017-01-01T00:00:00.000Z"
end_time = "2017-12-31T23:59:59.000Z"

import time

#print(list_AuthorID[6:7])
for idx, element_AuthorID in enumerate(list_AuthorID[20484 + 1:]):
  print(idx + 20484+ 1, "：", element_AuthorID)
  FileName=KEYWORDS_FileName+"_"+str(idx+ 20484 + 1)
  Pull_tweets_from_specific_users(element_AuthorID, start_time, end_time, max_results_author, next_token, OUTPUT_DIR_RESULTS,  FileName)

  time.sleep(1)

In [ ]:
print(list_AuthorID.index(720287831331987456))

20460


#Z. デバック用

In [ ]:
list_AuthorID.sort()
print(list_AuthorID)

In [ ]:
with open('/content/drive/MyDrive/sample.txt') as f:
  list_AuthorID=f.readlines()
  for i in range(len(list_AuthorID)):
    list_AuthorID[i]=list_AuthorID[i].rstrip()
  print(list_AuthorID)

In [ ]:
next_token=None
max_results_author=100
start_time = "2017-01-01T00:00:00.000Z"
end_time = "2017-12-31T23:59:59.000Z"

#print(list_AuthorID[6:7])
for idx, element_AuthorID in enumerate(list_AuthorID[0:9]):
  print(idx, "：", element_AuthorID)
  FileName=KEYWORDS_FileName+"_"+str(idx)
  Pull_tweets_from_specific_users(element_AuthorID, start_time, end_time, max_results_author, next_token, OUTPUT_DIR_RESULTS,  FileName)